In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

Loading and concatenating the word2vec tensor and GoogleNet representation generated for our dataset which will serve as input features to the MLP

In [2]:
wv  = torch.load("./dataset/word2vec_emb_tensor.pkl")

In [3]:
cnn = torch.load("./dataset/googleNet_cnn_emb_tensor.pkl")

In [4]:
X = torch.cat((wv,cnn),dim=1)

In [5]:
df = pd.read_csv("./dataset/books_with_genres.csv")

Setting the output labels for the classification of genre into fiction/non-fiction

In [6]:
df["fiction_binary"] = np.where(df['binary_category']=='fiction', 1, 0)

In [7]:
y = df["fiction_binary"]

In [8]:
y = y.to_numpy()
y = torch.from_numpy(y)

In [9]:
X_train = X[[i for i in range(4000)]]
X_test = X[[i for i in range(4000,5000)]]
y_train = y[[i for i in range(4000)]]
y_test = y[[i for i in range(4000, 5000)]]

In [10]:
y_train=y_train.long()
y_test=y_test.long()

Defining the MLP module and running it on GPU

In [11]:
device = torch.device("cuda")

In [12]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu1 = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [13]:
input_size = wv.shape[1]+cnn.shape[1]

In [14]:
net = MLP(input_size,2,512)
net = net.to(device)

Using CrossEntropyLoss for calculation of loss and Adam for optimisation

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.Adam( net.parameters() , lr=5e-4 )

Training the model

In [16]:
import torch.optim as optim
from random import randint

bs = 1

for epoch in range(50):

    for iter in range(1,500):
        # create a minibatch
        indices=torch.LongTensor(bs).random_(0,4000)
        minibatch_data =  X_train[indices].to(device)
        minibatch_label = y_train[indices].to(device)

        #reshape them to fit the network
        inputs=minibatch_data.view(bs,1100)

        inputs.requires_grad_()
        prob=net(inputs)

        loss = criterion(prob, minibatch_label)    
        optimizer.zero_grad()       
        loss.backward()
        optimizer.step()

Testing the model and calculating f1, accuracy, precision and recall

In [17]:
y_pred = []
y_true = []

bs = 1

for i in range(0,1000,bs):

    # extract the minibatch
    minibatch_data =  X_test[i:i+bs].to(device)
    minibatch_label= y_test[i:i+bs].to(device)

    # reshape the minibatch
    inputs = minibatch_data.view(bs,input_size)

    with torch.set_grad_enabled(False):
    # feed it to the network
        scores=net( inputs ) 
        scores = scores.argmax(dim=1)
        scores = scores.long()
        y_pred.append(scores.to('cpu'))
        y_true.append(minibatch_label.to('cpu'))


In [18]:
true = []
for i in y_true:
    true.append(i.item())

In [19]:
pred = []
for i in y_pred:
    pred.append(i.item())

In [20]:
total_f1 = f1_score(pred,true)
total_accuracy = accuracy_score(pred,true)
total_precision = precision_score(pred,true)
total_recall = recall_score(pred,true)

print( 'f1 score on test set  =', total_f1)
print( 'accuracy score on test set  =', total_accuracy)
print( 'precision score on test set  =', total_precision)
print( 'recall score on test set  =', total_recall)

f1 score on test set  = 0.7748478701825559
accuracy score on test set  = 0.667
precision score on test set  = 0.8995290423861853
recall score on test set  = 0.6805225653206651
